# APIM ❤️ AI Agents

## Model Context Protocol (MCP) lab
![flow](../../images/model-context-protocol.gif)

Playground to experiment the [Model Context Protocol](https://modelcontextprotocol.io/) with Azure API Management to enable plug & play of tools to LLMs. Leverages the [credential manager](https://learn.microsoft.com/en-us/azure/api-management/credentials-overview) for  managing OAuth 2.0 tokens to backend tools and [client token validation](https://learn.microsoft.com/en-us/azure/api-management/validate-jwt-policy) to ensure end-to-end authentication and authorization.  

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}-servicenow-test" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

build = 0
github_mcp_server_image = "github-mcp-server"
github_mcp_server_src = "src/github/mcp-server"

weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "src/weather/mcp-server"

servicenow_mcp_server_image = "servicenow-mcp-server"
service_mcp_server_src = "src/servicenow/mcp-server"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 12:08:33.715387 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    github_containerapp_resource_name = utils.get_deployment_output(output, 'gitHubMCPServerContainerAppResourceName', 'GitHub Container App Resource Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    servicenow_containerapp_resource_name = utils.get_deployment_output(output, 'servicenowMCPServerContainerAppResourceName', 'servicenow Container App Resource Name')


<a id='4'></a>
### 4️⃣ Build and deploy the MCP Servers



In [ ]:
build = build + 1 # increment the build number
utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
          "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
          "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

utils.run(f"az acr build --image {github_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {github_mcp_server_src}/Dockerfile {github_mcp_server_src}/. --no-logs", 
          "GitHub MCP Server image was successfully built", "Failed to build the GitHub MCP Server image")
utils.run(f'az containerapp update -n {github_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{github_mcp_server_image}:v0.{build}"', 
          "GitHub MCP Server deployment succeeded", "GitHub MCP Server deployment failed")

utils.run(f"az acr build --image {servicenow_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {servicenow_mcp_server_src}/Dockerfile {servicenow_mcp_server_src}/. --no-logs", 
          "ServiceNow MCP Server image was successfully built", "Failed to build the ServiceNow MCP Server image")
utils.run(f'az containerapp update -n {servicenow_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{servicenow_mcp_server_image}:v0.{build}"', 
          "ServiceNow MCP Server deployment succeeded", "ServiceNow MCP Server deployment failed")


<a id='testconnection'></a>
### 🧪 Test the connection to the Weather MCP server and List Tools

👉 Run `pip install mcp` before executing the cell

In [ ]:
import os, json, asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url):
    async with sse_client(server_url) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()

            response = await session.list_tools()
            tools = response.tools
    print(f"✅ Connected to server {server_url}")
    print("⚙️ Tools:")
    for tool in tools:
        print(f"  - {tool.name}")
        print(f"     Input Schema: {tool.inputSchema}")

asyncio.run(list_tools(f"{apim_resource_gateway_url}/weather/sse"))
asyncio.run(list_tools(f"{apim_resource_gateway_url}/github/sse"))
asyncio.run(list_tools(f"{apim_resource_gateway_url}/servicenow/sse"))


<a id='inspector'></a>
### 🧪 (optional) Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for testing and debugging the MCP servers

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Open the provided URL in a browser
3. Set the transport type as SSE
4. Provide the MCP server url and click connect
5. Select the "Tools" tab to see and run the available tools

<a id='functioncalling'></a>
### 🧪 Run an OpenAI completion with MCP tools

👉 Both the calls to Azure OpenAI and the MCP tools will be managed through Azure API Management.  


In [ ]:
# type: ignore
import json, asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client
from openai import AzureOpenAI
import nest_asyncio
nest_asyncio.apply()

async def call_tool(mcp_session, function_name, function_args):
    try:
        func_response = await mcp_session.call_tool(function_name, function_args)
        func_response_content = func_response.content
    except Exception as e:
        func_response_content = json.dumps({"error": str(e)})
    return str(func_response_content)

async def run_completion_with_tools(server_url, prompt):
    async with sse_client(server_url) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()
            response = await session.list_tools()
            tools = response.tools
            print(f"✅ Connected to server {server_url}")
            openai_tools = [{
                    "type": "function",
                    "function": {
                        "name": tool.name,
                        "parameters": tool.inputSchema
                    },
                } for tool in tools]

            # Step 1: send the conversation and available functions to the model
            print("▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt")
            client = AzureOpenAI(
                azure_endpoint=apim_resource_gateway_url,
                api_key=apim_subscription_key,
                api_version=openai_api_version
            )            
            messages = [ {"role": "user", "content": prompt} ]
            response = client.chat.completions.create(
                model=openai_deployment_name,
                messages=messages,
                tools=openai_tools,
            )
            response_message = response.choices[0].message
            tool_calls = response_message.tool_calls
            if tool_calls:
                # Step 2: call the function
                messages.append(response_message)  # extend conversation with assistant's reply
                # send the info for each function call and function response to the model
                print("▶️ Step 2: call the functions")
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    print(f"   Function Name: {function_name} Function Args: {function_args}")

                    function_response = await call_tool(session, function_name, function_args)
                    # Add the tool response
                    print(f"   Function response: {function_response}")
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": function_response,
                        }
                    )  # extend conversation with function response
                print("▶️ Step 3: finish with a completion to anwser the user prompt using the function response")
                second_response = client.chat.completions.create(
                    model=openai_deployment_name,
                    messages=messages,
                )  # get a new response from the model where it can see the function response
                print("💬", second_response.choices[0].message.content)

asyncio.run(run_completion_with_tools(f"{apim_resource_gateway_url}/weather/sse", "What's the current weather in Lisbon?"))


<a id='autogen'></a>
### 🧪 Execute an [AutoGen Agent using MCP Tools](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.tools.mcp.html) via Azure API Management


👉 Run `pip install autogen-core autogen-ext[openai,azure,mcp] autogen-agentchat` before executing the cell

In [ ]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.mcp import SseMcpToolAdapter, SseServerParams, mcp_server_tools
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

async def run_agent(url, prompt) -> None:
    # Create server params for the remote MCP service
    server_params = SseServerParams(
        url=url,
        headers={"Content-Type": "application/json"},
        timeout=30,  # Connection timeout in seconds
    )

    # Get all available tools
    tools = await mcp_server_tools(server_params)

    # Create an agent that can use the translation tool
    model_client = AzureOpenAIChatCompletionClient(azure_deployment=openai_deployment_name, model=openai_deployment_name,
                azure_endpoint=apim_resource_gateway_url,
                api_key=apim_subscription_key,
                api_version=openai_api_version
    )
    agent = AssistantAgent(
        name="weather",
        model_client=model_client,
        tools=tools, # type: ignore
        system_message="You are a helpful assistant.",
    )
    await Console(
        agent.run_stream(task=prompt)
    )

asyncio.run(run_agent(f"{apim_resource_gateway_url}/weather/sse", "What's the weather in Lisbon?"))


<a id='githubconfig'></a>
### 5️⃣ Create a GitHub OAuth app and configure the credential provider

#### Step 1 - [Register the application in GitHub](https://learn.microsoft.com/en-us/azure/api-management/credentials-how-to-github#step-1-register-an-application-in-github)

👉 Use the Authorization callback URL that is provided bellow  
👉 Copy the Client ID and Client secret

#### Step 2 - [Configure the credential provider in API Management](https://learn.microsoft.com/en-us/azure/api-management/credentials-how-to-github#step-2-configure-a-credential-provider-in-api-management)

👉 You just need to update the Client ID and Client secret on the existing `github` credential manager provider  
👉 Disregard the remaining steps outlined in the documentation

In [ ]:
print(f"Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/{apim_resource_name}")

<a id='githubtest'></a>
### 🧪 Run the GitHub MCP Server with VS Code or Cursor to retrieve GitHub Issues

1. Add the GitHub MCP Server
2. Type in the chat the following prompt: `Please list all the issues assigned to me in the GitHub repository {your-repo-name} under the organization {your-org-name}.`
3. The agent will suggest running the `authorize_github` tool.
4. Once the user accepts to run the tool, the agent will call the `authorize_github` and provide an URL to proceed with the authentication and authorization on GitHub.
5. After the user confirms that it's done, the agent will suggest running the `get_user` tool.
6. Once the user accepts to run the `get_user` tool, the agent will call the tool, return user information as context and suggest running the `get_issues` tool.
7. Once the user accepts to run the `get_issues` tool, the agent will provide the list of issues from the given repo.

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.